In [3]:
import MetaTrader5 as mt5
import pandas as pd
from time import sleep
from joblib import load
import ta  # For indicators like RSI, MACD, etc.

In [4]:
if not mt5.initialize():
    print("MetaTrader5 initialization failed")
    mt5.shutdown()
else:
    print("MetaTrader5 initialized successfully")

In [ ]:
account_info = mt5.account_info()
if account_info is not None:
    print(f"Account Info:\n{account_info}")
else:
    print("Failed to retrieve account info")

In [7]:
# Load Pre-trained Random Forest Model
model = load("../random_forest_forex_model.pkl")  # Replace with your model path


In [8]:
# Define symbol and parameters
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1
num_bars = 50  # Minimum data required for feature calculation


In [12]:
def add_indicators(df):
    # Exponential Moving Averages
    df['EMA_5'] = df['Close'].ewm(span=5, adjust=False).mean()
    df['EMA_15'] = df['Close'].ewm(span=15, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()

    # Simple Moving Averages
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['SMA_15'] = df['Close'].rolling(window=15).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()

    # Bollinger Bands
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Middle'] + 2 * df['Close'].rolling(window=20).std()
    df['BB_Lower'] = df['BB_Middle'] - 2 * df['Close'].rolling(window=20).std()

    # True Range and Average True Range
    df['TR'] = abs(df['High'] - df['Low'])
    df['ATR'] = df['TR'].rolling(window=14).mean()
    df['ATR_14'] = df['ATR']

    # Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # Moving Average Convergence Divergence (MACD)
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Average Directional Index (ADX)
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window=14).mean()
    plus_dm = (df['High'] - df['High'].shift(1)).where((df['High'] - df['High'].shift(1)) > (df['Low'].shift(1) - df['Low']), 0)
    minus_dm = (df['Low'].shift(1) - df['Low']).where((df['Low'].shift(1) - df['Low']) > (df['High'] - df['High'].shift(1)), 0)
    plus_di = 100 * (plus_dm.rolling(window=14).sum() / df['ATR_14'])
    minus_di = 100 * (minus_dm.rolling(window=14).sum() / df['ATR_14'])
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
    df['ADX_14'] = dx.rolling(window=14).mean()

    # Stochastic Oscillator
    df['Stochastic_%K'] = ((df['Close'] - df['Low'].rolling(window=14).min()) /
                           (df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min())) * 100
    df['Stochastic_%D'] = df['Stochastic_%K'].rolling(window=3).mean()

    # Williams %R
    df['Williams_%R'] = ((df['High'].rolling(window=14).max() - df['Close']) /
                         (df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min())) * -100

    # Chandelier Exit
    df['Chandelier_Exit_Long'] = df['High'].rolling(window=22).max() - (3 * df['ATR_14'])
    df['Chandelier_Exit_Short'] = df['Low'].rolling(window=22).min() + (3 * df['ATR_14'])

    # Volume Weighted Average Price (VWAP)
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()

    return df


In [13]:
# Fetch Live Data Function
def fetch_live_data(symbol, timeframe, num_bars):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    if rates is None:
        print(f"Failed to fetch data for {symbol}: {mt5.last_error()}")
        return None
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')  # Convert timestamp to datetime
    return data


In [ ]:

# Main Prediction Loop
try:
    while True:
        # Step 1: Fetch live data
        data = fetch_live_data(symbol, timeframe, num_bars)
        if data is None:
            print("Skipping prediction due to data fetch error.")
            sleep(60)
            continue
        
        # Step 2: Preprocess and add indicators
        data = data.rename(columns={"time": "DateTime", "open": "Open", "high": "High", 
                                     "low": "Low", "close": "Close", "tick_volume": "Volume"})
        data = add_indicators(data)
        data.dropna(inplace=True)  # Drop rows with missing values
        
        # Step 3: Prepare features for prediction
        features = ['EMA_5', 'EMA_15', 'BB_Middle', 'BB_Upper', 'BB_Lower', 
                    'MACD', 'MACD_Signal']  # Include the features used in model training
        X = data[features].iloc[-1:]  # Use the most recent row for prediction
        
        # Step 4: Predict with the Random Forest model
        prediction = model.predict(X)
        decision = "Buy" if prediction[0] == 1 else "Sell"
        print(f"Prediction for {symbol}: {decision}")
        
        # Wait for the next 1-minute candle
        sleep(60)
        
except KeyboardInterrupt:
    print("Bot stopped manually")
finally:
    mt5.shutdown()



Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EURUSD: (-10004, 'No IPC connection')
Skipping prediction due to data fetch error.
Failed to fetch data for EUR